#### I run this notebook in Kaggle because it does not recognize deep_translator here after installation
#### I did the translation both for both:
####   - MultiLabelSongs_EN.csv => MultiLabelSongs_RO.csv
####   - MultiLabelSongs_EN_prepared.csv => MultiLabelSongs_RO_prepared.csv

In [2]:
!pip install deep_translator


[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import pandas as pd
from tqdm import tqdm
from deep_translator import GoogleTranslator
import time
import re

ModuleNotFoundError: No module named 'deep_translator'

In [ ]:
songs_df = pd.read_csv('../input/en-datasets/MultiLabelSongs_EN_prepared.csv')
sample_df = songs_df.copy()
translator = GoogleTranslator(source='auto', target='ro')

In [ ]:
def chunk_text(text, max_len=4000):
    text = str(text)
    if len(text) <= max_len:
        return [text]

    chunks, start = [], 0
    while start < len(text):
        end = start + max_len
        if end >= len(text):
            chunks.append(text[start:])
            break

        split_pos = max(
            text.rfind('.', start, end),
            text.rfind('!', start, end),
            text.rfind('?', start, end),
            text.rfind('\n', start, end),
            text.rfind(' ', start, end)
        )
        if split_pos <= start:
            split_pos = end

        chunks.append(text[start:split_pos].strip())
        start = split_pos + 1

    return chunks

def translate_text(text, max_retries=3):
    text = str(text)
    chunks = chunk_text(text, max_len=4000)
    translated_chunks = []
    for chunk in chunks:
        for attempt in range(max_retries):
            try:
                translated = translator.translate(chunk)
                translated_chunks.append(translated)
                break
            except Exception as e:
                print(f"Error at translation, attempt {attempt + 1}/{max_retries}: {e}")
                time.sleep(1)
        else:
            print(f"Non-translated even after {max_retries} tries: {chunk[:30]}...")
            translated_chunks.append(chunk)

    return " ".join(translated_chunks)

def needs_translation(row):
    original = songs_df.loc[row.name, 'lyrics']
    return row['lyrics'] == original

In [ ]:
column = 'lyrics'
max_passes = 5

for pass_num in range(1, max_passes + 1):
    print(f"\n Pass {pass_num} to translate")
    tqdm.pandas(desc=f"Translating pass {pass_num}")
    mask = sample_df.apply(needs_translation, axis=1)
    if not mask.any():
        print("Translated all")
        break

    sample_df.loc[mask, column] = sample_df.loc[mask, column].progress_apply(translate_text)
    sample_df.to_csv('../translated-temporary.csv', index=False)
    print(f"Saved temporary backup after pass {pass_num}")

else:
    print("Maximum number of tries reached")

not_translated = sample_df[sample_df[column] == songs_df[column]]
print(f"\nTotal number of rows not translated: {len(not_translated)}")

In [ ]:
def correct_hyphens(text):
    return re.sub(r'\b(\w+)\s*-\s*(\w+)\b', r'\1-\2', text)

sample_df[column] = sample_df[column].apply(correct_hyphens)
sample_filename = 'MultiLabelSongs_RO_prepared.csv'
sample_df.to_csv(sample_filename, index=False)

Număr rânduri netraduse la final: 9
Exemple de rânduri netraduse:
516     They seek him here they seek him there His clo...
537     - what condition my condition was in I woke up...
591     Cause he gets up in the morning And he goes to...
620     Dry lightning cracks across the skies Those st...
779     That there that is not me I go where I please ...
880     Puts up the closed sign does the man in the co...
897     there is only one chance in a million Of a dre...
969     I ’ ll drown my beliefs To have your babies I ...
1017    we are having a party I hope you are hearty we...